In [1]:
import tensorflow as tf
mnist = tf.keras.datasets.mnist
from tensorflow.keras import Sequential,Model
import tensorboard

C:\Users\meakh\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\meakh\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\meakh\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\meakh\AppData\Roaming\Python\Python37\site-

In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [3]:
X_train = X_train.reshape(60000,28,28,1)
X_test = X_test.reshape(10000,28,28,1)

In [4]:
X_train = X_train/255
X_test = X_test/255

In [5]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='softmax')
])

In [6]:
model.compile(optimizer= 'adam', loss= 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [10]:
import os
log_dir = os.path.join('logs')
tensor_board = tf.keras.callbacks.TensorBoard(log_dir = log_dir,histogram_freq=1, profile_batch = 100000000)

In [20]:
with tf.device('/gpu:0'):
    model.fit(X_train, y_train, batch_size=512, epochs=15, verbose=2,
              validation_data=(X_test, y_test),
              callbacks=[tensor_board])

Train on 60000 samples, validate on 10000 samples
Epoch 1/15
60000/60000 - 6s - loss: 0.0447 - accuracy: 0.9864 - val_loss: 0.0307 - val_accuracy: 0.9901
Epoch 2/15
60000/60000 - 6s - loss: 0.0420 - accuracy: 0.9870 - val_loss: 0.0311 - val_accuracy: 0.9902
Epoch 3/15
60000/60000 - 6s - loss: 0.0371 - accuracy: 0.9886 - val_loss: 0.0331 - val_accuracy: 0.9892
Epoch 4/15
60000/60000 - 6s - loss: 0.0362 - accuracy: 0.9886 - val_loss: 0.0285 - val_accuracy: 0.9911
Epoch 5/15
60000/60000 - 6s - loss: 0.0312 - accuracy: 0.9902 - val_loss: 0.0301 - val_accuracy: 0.9909
Epoch 6/15
60000/60000 - 6s - loss: 0.0306 - accuracy: 0.9903 - val_loss: 0.0299 - val_accuracy: 0.9909
Epoch 7/15
60000/60000 - 6s - loss: 0.0297 - accuracy: 0.9904 - val_loss: 0.0288 - val_accuracy: 0.9919
Epoch 8/15
60000/60000 - 6s - loss: 0.0255 - accuracy: 0.9919 - val_loss: 0.0319 - val_accuracy: 0.9908
Epoch 9/15
60000/60000 - 6s - loss: 0.0242 - accuracy: 0.9915 - val_loss: 0.0337 - val_accuracy: 0.9911
Epoch 10/15
60

In [22]:
model.save_weights('weights')

In [23]:
model.load_weights('weights')

In [33]:
model.save('model')

In [5]:
model = tf.keras.models.load_model('model')

In [12]:
with tf.device('/gpu:0'):
    model.fit(X_train, y_train, batch_size=512, epochs=1, verbose=1,
              validation_data=(X_test, y_test),
              callbacks=[tensor_board])

Train on 60000 samples, validate on 10000 samples
60000/60000 - 6s - loss: 0.0140 - accuracy: 0.9951 - val_loss: 0.0332 - val_accuracy: 0.9926


In [ ]:
EPOCHS = 10
checkpoint_filepath = '/tmp/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_acc',
    mode='max',
    save_best_only=True)

# Model weights are saved at the end of every epoch, if it's the best seen
# so far.
model.fit(epochs=EPOCHS, callbacks=[model_checkpoint_callback])

# The model weights (that are considered the best) are loaded into the model.
model.load_weights(checkpoint_filepath)

In [25]:
path = 'abcd'
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath = path,
                                                     monitor = 'val_accuracy',
                                                     save_weights_only = True,
                                                     mode = 'max',
                                                     save_best_only = True)
with tf.device('/gpu:0'):
    model.fit(X_train, y_train, batch_size=512, epochs=1, verbose=1,
              validation_data=(X_test, y_test),
              callbacks=[tensor_board, model_checkpoint])


Train on 60000 samples, validate on 10000 samples
60000/60000 [==============================] - 6s 105us/sample - loss: 0.0107 - accuracy: 0.9964 - val_loss: 0.0357 - val_accuracy: 0.9925


In [26]:
model.load_weights(path)

In [36]:
import os
import datetime
log_dir = "logs/fit/"
tensor_board = tf.keras.callbacks.TensorBoard(log_dir = log_dir,histogram_freq=1, profile_batch = 100000000)
with tf.device('/gpu:0'):
    model.fit(X_train, y_train, batch_size=512, epochs=1, verbose=1,
              validation_data=(X_test, y_test),
              callbacks=[tensor_board, model_checkpoint])

Train on 60000 samples, validate on 10000 samples
60000/60000 [==============================] - 6s 101us/sample - loss: 0.0099 - accuracy: 0.9966 - val_loss: 0.0338 - val_accuracy: 0.9929


In [ ]:
tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=0,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
)

In [59]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=0)

In [50]:
with tf.device('/gpu:0'):
    model.fit(X_train, y_train, batch_size=512, epochs=15, verbose=1,
              validation_data=(X_test, y_test),
              callbacks=[tensor_board, early_stop])

Train on 60000 samples, validate on 10000 samples
Epoch 1/15
60000/60000 [==============================] - 6s 98us/sample - loss: 0.0080 - accuracy: 0.9972 - val_loss: 0.0353 - val_accuracy: 0.9920
Epoch 2/15
60000/60000 [==============================] - 6s 99us/sample - loss: 0.0076 - accuracy: 0.9975 - val_loss: 0.0361 - val_accuracy: 0.9929
Epoch 3/15
60000/60000 [==============================] - 6s 99us/sample - loss: 0.0082 - accuracy: 0.9974 - val_loss: 0.0348 - val_accuracy: 0.9932


In [55]:
import math
def scheduler(epoch,lr):
        return lr*math.exp(-0.1)
    
lr_call = tf.keras.callbacks.LearningRateScheduler(scheduler)
with tf.device('/gpu:0'):
    model.fit(X_train, y_train, batch_size=512, epochs=15, verbose=1,
              validation_data=(X_test, y_test),
              callbacks=[tensor_board, lr_call, early_stop])
round(model.optimizer.lr.numpy(), 5)

Train on 60000 samples, validate on 10000 samples
Epoch 1/15
60000/60000 [==============================] - 7s 124us/sample - loss: 0.0064 - accuracy: 0.9978 - val_loss: 0.0363 - val_accuracy: 0.9929
Epoch 2/15
60000/60000 [==============================] - 6s 100us/sample - loss: 0.0058 - accuracy: 0.9980 - val_loss: 0.0367 - val_accuracy: 0.9938
Epoch 3/15
60000/60000 [==============================] - 6s 100us/sample - loss: 0.0055 - accuracy: 0.9982 - val_loss: 0.0370 - val_accuracy: 0.9929
Epoch 4/15
60000/60000 [==============================] - 6s 100us/sample - loss: 0.0045 - accuracy: 0.9984 - val_loss: 0.0403 - val_accuracy: 0.9932
Epoch 5/15
60000/60000 [==============================] - 6s 100us/sample - loss: 0.0049 - accuracy: 0.9984 - val_loss: 0.0351 - val_accuracy: 0.9935


0.00061

In [63]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='accuracy', factor=0.2,
                              patience=0)

csv_logger = tf.keras.callbacks.CSVLogger('training.log')

with tf.device('/gpu:0'):
    model.fit(X_train, y_train, batch_size=512, epochs=15, verbose=1,
              validation_data=(X_test, y_test),
              callbacks=[tensor_board, reduce_lr, csv_logger])
round(model.optimizer.lr.numpy(), 20)

Train on 60000 samples, validate on 10000 samples
Epoch 1/15
60000/60000 [==============================] - 6s 100us/sample - loss: 0.0032 - accuracy: 0.9990 - val_loss: 0.0376 - val_accuracy: 0.9934
Epoch 2/15
59392/60000 [============================>.] - ETA: 0s - loss: 0.0033 - accuracy: 0.9988
Epoch 00002: ReduceLROnPlateau reducing learning rate to 6.359935634802584e-18.
60000/60000 [==============================] - 6s 99us/sample - loss: 0.0033 - accuracy: 0.9988 - val_loss: 0.0376 - val_accuracy: 0.9934
Epoch 3/15
59904/60000 [============================>.] - ETA: 0s - loss: 0.0039 - accuracy: 0.9986
Epoch 00003: ReduceLROnPlateau reducing learning rate to 1.271987143504129e-18.
60000/60000 [==============================] - 6s 99us/sample - loss: 0.0039 - accuracy: 0.9987 - val_loss: 0.0376 - val_accuracy: 0.9934
Epoch 4/15
59904/60000 [============================>.] - ETA: 0s - loss: 0.0031 - accuracy: 0.9990
Epoch 00004: ReduceLROnPlateau reducing learning rate to 2.54397

0.0